In [52]:
import os

In [53]:
pwd

'd:\\vscode\\Text_Classification'

In [3]:
os.chdir("../")

In [54]:
pwd

'd:\\vscode\\Text_Classification'

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionsConfig:
    root_dir :Path
    source_URL:str
    local_data_file : Path
    unzip_dir:Path


In [56]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [57]:
class ConfigureManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories ([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionsConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionsConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )


In [58]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [59]:
class DataIngestion:
    def __init__(self, config: DataIngestionsConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [60]:
try:
    config = ConfigureManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-07 12:22:54,741: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-07 12:22:54,747: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-07 12:22:54,749: INFO: common: created directory at: artifacts]
[2024-03-07 12:22:54,752: INFO: common: created directory at: artifacts/data_ingestion]


AttributeError: 'NoneType' object has no attribute 'local_data_file'